# 第12章　発展編：機械学習を用いたアプタマー配列の解析と創薬

 - 岩野 夏樹
 - 浜田 道昭

編集部注：2023年5月29日最終更新．コードの一部がお手元の書籍と異なる可能性がございます．正誤・更新情報は弊社ウェブサイトの[本書詳細ページ](https://www.yodosha.co.jp/jikkenigaku/book/9784758122634/index.html)をご参照ください．

##### 入力12-1

In [ ]:
# 依存関係のインストールを行う
!python -m pip install git+https://github.com/hmdlab/raptgen.git@v1.0.0
!sudo apt-get install fonts-noto-cjk
!pip install biopython CairoSVG svgwrite GPy # 更新いたしました
!pip install git+https://github.com/SheffieldML/GPyOpt.git # 更新いたしました

##### 入力12-2

In [ ]:
# 本ノートブックには重いモデルの学習の処理がある
# モデルの学習の処理をスキップして，すでに学習済みのモデルを読み込む場合はskip_heacy_computationをTrueにする
# スキップせず，学習を行う場合にはFalseにする
skip_heavy_computation = True

if skip_heavy_computation:
    reload_model = True
else:
    reload_model = False

##### 入力12-3

In [ ]:
# 今回作成したシミュレーションファイルをダウンロードする
!wget https://raw.githubusercontent.com/hmdlab/raptgen/v1.0.0/data/simulation/paired/sequences.txt

##### 入力12-4

In [ ]:
# ダウンロードしたファイルの先頭の10行を表示する
# ファイルはモチーフの番号, 配列の形式で保存されている
!head sequences.txt

##### 入力12-5

In [ ]:
# まずは必要なライブラリをインポートする
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pathlib import Path

# シミュレーションで作成したファイルを読み込む
with Path("sequences.txt").open() as f:
    records = []
    for idx, line in enumerate(f.readlines()):
        # 行は 1,ATGCATGC のようにモチーフのIDと配列で区切られているので，その部分を処理する
        motif_idx, sequence = line.strip().split(",")

        # SeqIOで出力可能なSeqRecord形式に変換する
        record = SeqRecord(
            Seq(sequence),
            id=f"{idx}",
            description=f"motif:{motif_idx} #{idx}"
        )

        # リストに追加する
        records.append(record)

    # 配列をFASTA形式で保存する
    SeqIO.write(
        sequences = records,
        handle = "sample.fasta",
        format = "fasta"
    )

##### 入力12-6

In [ ]:
# 生成したFASTAファイル
!head -n 4 sample.fasta

##### 入力12-7


In [ ]:
# FASTA形式のファイルを読み込む
from raptgen.data import SingleRound
filename = "sample.fasta"
single_round = SingleRound(path = filename)

##### 入力12-8

In [ ]:
import torch

# 以下はGoogle Colaboratory上ではスライドバーで表示される
epochs = 200 #@param {type:"slider", min:0, max:500, step:5}
threshold = 10 #@param {type:"slider", min:0, max:100, step:1}
device = "cuda" #@param ["cuda", "cpu"]

# GPUが使えない場合にはcudaと設定されていてもCPUを使うように設定する
if device == "cuda" and torch.cuda.is_available() == False:
    print("cuda is not available. using cpu.")
    device = torch.device("cpu")
else:
    print(f"using {device}.")
    device = torch.device(device)

min_count = 1 #@param {type:"slider", min:0, max:10, step:1}

# 学習のために必要なDataLoaderはSingleRoundクラスのget_dataloader関数で取得できる
# RaptGenの学習速度は学習配列が短いほど速いので，アダプター部分を切り落としたデータが必要になるが，
# この関数はその処理も内部で行っている
train_loader, test_loader = single_round.get_dataloader(
    min_count=min_count, use_cuda=(device == torch.device("cuda"))
)
force_epochs = 10 #@param {type:"slider", min:0, max:100, step:1}
save_dir = "out/" #@param {type:"string"}
save_dir = Path(save_dir).expanduser()
save_dir.mkdir(exist_ok = True, parents=True)

##### 入力12-9

In [ ]:
# エンコーダはCNNベース，デコーダは潜在表現をプロファイルHMMの確率分布のパラメータに復元するVAEを利用する
from raptgen.models import CNN_PHMM_VAE
model = CNN_PHMM_VAE(
    motif_len=single_round.random_region_length, # モデルサイズとしてSELEXのランダム領域と同じ大きさを利用する
    embed_size=2 # 埋め込みの次元は2次元とする
)
model = model.to(device)

# 学習のためのパラメータの更新にはAdamを利用する
from torch import optim
optimizer = optim.Adam(model.parameters())

# 学習のパラメータを設定する
train_kwargs = {
    "epochs" : epochs,
    "threshold" : threshold,
    "device" : device,
    "train_loader" : train_loader,
    "test_loader" : test_loader,
    "save_dir" : save_dir,
    "beta_schedule" : True,
    "force_matching" : True,
    "force_epochs" : force_epochs,
    "model" : model,
    "optimizer" : optimizer,
    "model_str" : "cnn_phmm_vae.mdl",
}

##### 入力12-10

In [ ]:
# 自分のデータなどで学習する場合は下記のコードを実行する
# シミュレーションデータではGPUを利用したうえで15分程度の時間が必要となる
from raptgen.models import train
if not skip_heavy_computation:
    train_result = train(**train_kwargs)

##### 入力12-11

In [ ]:
# 描画に必要なライブラリをインポートする
# ノートブックで見やすくなるように横長での出力をデフォルトとする
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10, 2.5]
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 150
mpl.rcParams['savefig.bbox'] = "tight"
from matplotlib import pyplot as plt
from raptgen.visualization import write_profile_hmm_svg
# 再現性のため，学習した結果のモデルを読み込む
# 配布時点では自分で学習したモデルを使用するのでFalseに設定してある
if reload_model:
    model = CNN_PHMM_VAE(
        motif_len=single_round.random_region_length,
        embed_size=2
    )

    # 学習済みモデルのパラメータのダウンロード
    !wget https://github.com/hmdlab/raptgen/raw/v1.0.0/results/simulation/paired/cnn_phmm_vae.mdl

    # モデルのパラメータのロード
    model.load_state_dict(torch.load("cnn_phmm_vae.mdl", map_location=device))

##### 入力12-12

In [ ]:
from raptgen.data import Result

# 学習結果の可視化のためにResultオブジェクトを作成する
result = Result(
    model.to("cpu"),
    experiment = single_round,
    path_to_save_results = save_dir,
    load_if_exists = True
)

# モデルの学習は配列のアダプター部分を切り落とした状態で行っているため，
# こちらでもそのアダプターを切り落とした表現を取得する
sequences = single_round.get_filter_passed_sequences(
    random_only = True
)

# embed_sequences関数で配列の潜在表現を得る
embed_x = result.embed_sequences(sequences)

##### 入力12-13

In [ ]:
# sequences.txtを開く
with Path("sequences.txt").open() as f:
    l = []
    for line in f.readlines():
        motif_idx_str, sequence = line.strip().split(",")
        l.append(int(motif_idx_str))
len(l)

##### 入力12-14

In [ ]:
# 埋め込みの結果の描画
fig, ax = plt.subplots()
# embed_xは[5000,2]のtorch.Tensorなので，これを2つの5000次元のベクトルに変換する.
x1, x2 = embed_x.T.detach().numpy()
# 凡例の表示用のラベル
labels = [
    "ACTAC(N*)CCCTC",
    "ACTAC(N*)",
    "(N*)CCCTC"
]
for i in range(3):
    # モチーフの番号に応じた系列の取り出し
    _x = [x for x, motif_id in zip(x1, l) if motif_id == i]
    _y = [x for x, motif_id in zip(x2, l) if motif_id == i]
    # 散布図で表示
    ax.scatter(
        _x,
        _y,
        label = labels[i]
    )
plt.xlabel("latent axis 1")
plt.ylabel("latent axis 2")
plt.legend()
plt.show()


##### 入力12-15

In [ ]:
# 潜在空間における特定の点のプロファイルHMMのモデルパラメータをSVGとして書き出す
# 今回は可視化のため，中心の配列を取り出して，そのパラメータを書き出す

# (x, y) = (0, 0)のデータを作成する
# 入力データの次元は (データ数, 軸数) なので，1×2のベクトルを作成する
x = torch.Tensor([[0, 0]])

# プロファイルHMMのパラメータをデコーダで返す
(a, e_m) = model.decoder(x)

# 0番目のデータを取り出して，torch.Tensor形式をnumpy.Array形式にする
a = a[0].detach().numpy()
e_m = e_m[0].detach().numpy()

# 可視化はa:遷移確率とe_m:出力確率とする
# svgで出力されるが，ノートブックに表示するためpngでも出力する
write_profile_hmm_svg(a, e_m, savepng=True)

# ノートブック内で表示
from PIL import Image
Image.open("profile_HMM.png")

##### 入力12-6

In [ ]:
from raptgen.visualization import draw_most_probable

# 最も確率の高い状態列の場合のシークエンスロゴの描画.^で表されているのがInsertion
draw_most_probable(a, e_m, force=True)

##### 入力12-17


In [ ]:
from raptgen.data import ProfileHMMSampler
sampler = ProfileHMMSampler(a, e_m, proba_is_log = True)

##### 入力12-18

In [ ]:
sampler.sample(sequence_only = True)

##### 入力12-19

In [ ]:
!wget https://raw.githubusercontent.com/hmdlab/raptgen/v1.0.0/data/real/A_4R.fastq
!wget https://raw.githubusercontent.com/hmdlab/raptgen/v1.0.0/results/real/A_best.mdl
!wget https://raw.githubusercontent.com/hmdlab/raptgen/v1.0.0/results/real/A_evaled.csv

##### 入力12-20

In [ ]:
# 実際のモデルとデータを利用する
experiment = SingleRound(path = "A_4R.fastq")
model = CNN_PHMM_VAE(experiment.random_region_length, embed_size=2)
model.load_state_dict(torch.load("A_best.mdl", map_location=device))
result = Result(
    model,
    experiment=experiment,
    path_to_save_results=save_dir,
    load_if_exists=True
)

##### 入力12-21

In [ ]:
import pandas as pd
df = pd.read_csv("A_evaled.csv", names=["seq", "activity"])
seq, act = df.values.T

result.evaluated_X = result.embed_sequences(seq)

# 最小化問題を解くため，スコアを正負逆転した値を用いる
# また，評価済みの配列の値は形状が[N, 1]となっている必要があるため，軸を増やす
result.evaluated_y = -act[:, None]

# すでに検査済みの配列の潜在表現と値を利用して，次に調査すべき潜在表現の位置を出力する
locations = result.get_bo_result(force_rerun=True)
locations

##### 入力12-22

In [ ]:
# 潜在空間上でベイズ最適化を行った状態の表示
# 等高線は推定された配列のスコアを表す
fig, ax = plt.subplots(figsize=(6,6))
result.plot_bo(ax, fig)
ax.set_xlabel("latent axis 1")
ax.set_ylabel("latent axis 2")
ax.legend()
plt.show()

##### 入力12-23

In [ ]:
# 配列はmodel.decoderでプロファイルHMMのパラメータからサンプリングしてもよいが，
# Resultクラスの関数で取得することができる
scores = result._points_to_score(torch.from_numpy(locations).float())

# scoreは配列のモチーフ，そこからサンプルされたモチーフ，その配列の対数確率，となる
df = pd.DataFrame(scores, columns = ["sequence_motif", "sampled_sequence", "log_ probability"])

# 推論結果の配列を表示する
df

##### 入力12-24

In [ ]:
# モチーフの長さを短くしたモデルを用意する
model_short = CNN_PHMM_VAE(
    motif_len=15, # 本来ランダム領域は20塩基だが，ここではモチーフの長さを15としている
    embed_size=2
)
model_short = model_short.to(device)
optimizer_short = optim.Adam(model_short.parameters())

# 学習のパラメータはモデルとoptimizerを除き同様である
train_kwargs = {
      "epochs" : epochs,
      "threshold" : threshold,
      "device" : device,
      "train_loader" : train_loader,
      "test_loader" : test_loader,
      "save_dir" : save_dir,
      "beta_schedule" : True,
      "force_matching" : True,
      "force_epochs" : force_epochs,
      "model" : model_short, # 変更した
      "optimizer" : optimizer_short,
      "model_str" : "cnn_phmm_vae.mdl",
}

##### 入力12-25

In [ ]:
from raptgen.models import train
train_result = train(**train_kwargs)

##### 入力12-26

In [ ]:
# 潜在空間における特定の点のプロファイルHMMのモデルパラメータをSVGとして書き出す
# 今回は可視化のため，中心の配列を取り出して，そのパラメータを書き出す

# (x, y) = (0, 0)のデータを作成する
# 入力データの次元は(データ数, 軸数)なので，1×2のベクトルを作成する
x = torch.Tensor([[-1, 0]])

# プロファイルHMMのパラメータをデコーダで返す
(a, e_m) = model_short.to("cpu").decoder(x)

# 0番目のデータを取り出して，torch.Tensor形式をnumpy.Array形式にする
a = a[0].detach().numpy()
e_m = e_m[0].detach().numpy()

# 埋め込みの結果の描画
fig, ax = plt.subplots()

result_short = Result(
    model_short.to("cpu"),
    experiment = single_round,
    path_to_save_results = save_dir,
    load_if_exists = True
)

embed_x = result_short.embed_sequences(sequences)
x1, x2 = embed_x.T.detach().numpy()

# 散布図を表示
ax.scatter(
    x1,
    x2,
    c = l,
    cmap = "tab20c"
)

plt.show()

##### 入力12-27

In [ ]:
# 可視化はa:遷移確率とe_m:出力確率とする
# svgで出力されるが，ノートブックに表示するためpngでも出力する
write_profile_hmm_svg(a, e_m, savepng=True, name="profile_HMM_short.svg")

# ノートブック内で表示
# 短鎖化されたモデルになっていることがわかる
Image.open("profile_HMM_short.png")